In [5]:
import runhouse as rh 

num_nodes = 3
cluster_name = f"py-{num_nodes}"

cluster = rh.ondemand_cluster(
        name=cluster_name,
        #instance_type= 'cpu:2+',
        memory = "12+",
        num_instances=num_nodes,
        provider="gcp",
    ).up_if_not()
from helper_dask import launch_dask_cluster
launch_dask_cluster(cluster)

try:
    while True:
        pass
except KeyboardInterrupt:
    print("Tore down Dask cluster")

INFO | 2024-11-11 16:13:00 | runhouse.resources.secrets.secret:285 | Saving config for ssh-sky-key to Den
INFO | 2024-11-11 16:13:00 | runhouse.resources.secrets.secret:301 | Saving secrets for ssh-sky-key to Vault
INFO | 2024-11-11 16:13:01 | runhouse.rns.rns_client:500 | Saving config to RNS: {'name': '/paul/py-3', 'resource_type': 'cluster', 'resource_subtype': 'OnDemandCluster', 'visibility': 'private', 'server_port': 32300, 'server_connection_type': 'ssh', 'den_auth': False, 'ssh_port': 22, 'ssh_properties': {}, 'creds': '/paul/ssh-sky-key', 'api_server_url': 'https://api.run.house', 'num_instances': 3, 'provider': 'gcp', 'open_ports': [], 'use_spot': False, 'memory': '12+', 'sky_kwargs': {}, 'launched_properties': {}, 'autostop_mins': -1}


Output()

INFO | 2024-11-11 16:13:06 | runhouse.resources.hardware.cluster:1746 | Running command on py-3: echo "hello"


In [1]:
from google.oauth2 import service_account
import gcsfs

# Load credentials
credentials = service_account.Credentials.from_service_account_file("/Users/paulyang/Downloads/runhouse-test-8c3eb368f020.json")

# Use credentials with GCSFS
gcs = gcsfs.GCSFileSystem(project="runhouse-test", token=credentials)


import dask.dataframe as dd
df = dd.read_parquet("gs://rh-demo-external/*.parquet", storage_options={"gcs": gcs})


/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/dask/dataframe/io/parquet/core.py:1249: FutureWarning: Could not find pyarrow; falling back to fastparquet, which is deprecated and will be removed in a future release.
  warnings.warn(


In [2]:
import runhouse as rh 
my_secret = rh.provider_secret(provider = "gcp", name = "runhouse-test-dataccess-serviceaccount", path = "/Users/paulyang/Downloads/runhouse-test-8c3eb368f020.json")

In [4]:
my_secret.save()

INFO | 2024-11-11 16:12:44 | runhouse.resources.secrets.secret:285 | Saving config for runhouse-test-dataccess-serviceaccount to Den
INFO | 2024-11-11 16:12:44 | runhouse.resources.secrets.secret:301 | Saving secrets for runhouse-test-dataccess-serviceaccount to Vault


In [3]:
my_secret.to()

INFO | 2024-11-11 16:12:40 | runhouse.rns.login:171 | Successfully logged into Runhouse.


In [ ]:
trainer = cluster.get("my_trainer", remote = True, default = None)
print(trainer.return_model_details())

INFO | 2024-11-06 22:54:24 | runhouse.resources.hardware.ssh_tunnel:250 | Port 32300 is already in use. Trying next port.
INFO | 2024-11-06 22:54:24 | runhouse.resources.hardware.ssh_tunnel:250 | Port 32301 is already in use. Trying next port.
INFO | 2024-11-06 22:54:24 | runhouse.resources.hardware.ssh_tunnel:250 | Port 32302 is already in use. Trying next port.
INFO | 2024-11-06 22:54:24 | runhouse.resources.hardware.ssh_tunnel:250 | Port 32303 is already in use. Trying next port.
INFO | 2024-11-06 22:54:24 | runhouse.resources.hardware.ssh_tunnel:250 | Port 32304 is already in use. Trying next port.
INFO | 2024-11-06 22:54:24 | runhouse.resources.hardware.ssh_tunnel:91 | Running forwarding command: ssh -T -L 32305:localhost:32300 -i ~/.ssh/sky-key -o Port=22 -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -o LogLevel=ERROR -o IdentitiesOnly=yes -o ExitOnForwardFailure=yes -o ServerAliveInterval=5 -o ServerAliveCountMax=3 -o ConnectTimeout=30s -o ForwardAgent=yes -o Contr

INFO:httpx:HTTP Request: GET http://127.0.0.1:32305/logs/my_trainer_return_model_details@20241106@225425_099342/pickle "HTTP/1.1 200 OK"


INFO | 2024-11-06 22:54:25 | runhouse.servers.http.http_client:503 | Time to call my_trainer.return_model_details: 0.04 seconds
{'features': ['passenger_count', 'trip_distance', 'fare_amount', 'day', 'month', 'dayofweek', 'hour'], 'mse': np.float64(9.26837997237001), 'mae': np.float64(1.8701559788491435)}


In [1]:
import runhouse as rh 
gcp_secret = rh.provider_secret("gcp", path="/Users/paulyang/Downloads/runhouse-minimal-permissions.json")

gcp_secret.save()


INFO | 2024-11-12 15:44:55 | runhouse.resources.secrets.secret:285 | Saving config for /paul/gcp to Den
INFO | 2024-11-12 15:44:55 | runhouse.resources.secrets.secret:301 | Saving secrets for /paul/gcp to Vault


In [9]:
num_nodes = 2
cluster_name = f"py-{num_nodes}-new-gcp"

# The environment for the remote cluster
env = rh.env(
    reqs=[
        "dask-ml",
        "dask[distributed]",
        "dask[dataframe]",
        "gcsfs",
        "lightgbm",
        "cloudpickle",
        "pyarrow==15.0.2"
    ],
)

cluster = rh.ondemand_cluster(
    name=cluster_name,
    instance_type= 'CPU:2+',
    memory = "12+",
    num_nodes=num_nodes,
    provider="gcp",
    region = "us-east1",
    launcher_type = "den", 
    env = env
).up_if_not()

#from helper_dask import launch_dask_cluster
#launch_dask_cluster(cluster)

import threading

def start_scheduler():
    cluster.run("dask scheduler --port 8786")

def start_worker(head_node_ip, node):
    cluster.run(f"dask worker tcp://{head_node_ip}:8786", node=node)

stable_ips = [ip[0] for ip in cluster.stable_internal_external_ips]
head_node_ip = stable_ips[0]

# Start the scheduler on the head node
scheduler_thread = threading.Thread(target=start_scheduler)
scheduler_thread.daemon = True
scheduler_thread.start()

# Start workers and connect to the scheduler
import time
print(cluster.ips)
for ip in cluster.ips:
    time.sleep(3)
    cluster.run('pip install "dask[distributed]"', node=ip)
    worker_thread = threading.Thread(target=start_worker, args=(head_node_ip, ip))
    worker_thread.daemon = True
    worker_thread.start()
    print('started ',ip)

from lightgbm_training import LightGBMModelTrainer
remote_dask_trainer = rh.module(LightGBMModelTrainer).to(cluster, env=env)

INFO | 2024-11-12 17:52:51 | runhouse.resources.secrets.provider_secrets.ssh_secret:90 | Secrets already exist in ~/.ssh/sky-key. Skipping.
INFO | 2024-11-12 17:52:51 | runhouse.resources.hardware.cluster:1747 | Running command on py-2-new-gcp: echo "hello"
INFO | 2024-11-12 17:52:53 | runhouse.resources.hardware.ssh_tunnel:238 | SSH tunnel on to server's port 32300 via server's ssh port 22 already created with the cluster.
INFO | 2024-11-12 17:52:53 | runhouse.servers.http.http_client:439 | Calling _cluster_default_env._run_command


INFO:httpx:HTTP Request: GET http://127.0.0.1:32304/logs/_cluster_default_env__run_command@20241112@175253_187716/pickle "HTTP/1.1 200 OK"


INFO | 2024-11-12 17:52:54 | runhouse.servers.http.http_client:503 | Time to call _cluster_default_env._run_command: 1.2 seconds
INFO | 2024-11-12 17:52:56 | runhouse.resources.hardware.cluster:1747 | Running command on py-2-new-gcp: pip install "dask[distributed]"
INFO | 2024-11-12 17:53:00 | runhouse.resources.hardware.cluster:1747 | Running command on py-2-new-gcp: dask worker tcp://10.142.0.63:8786
2024-11-12 22:53:03,447 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.142.0.63:39943'
INFO | 2024-11-12 17:53:03 | runhouse.resources.hardware.cluster:1747 | Running command on py-2-new-gcp: pip install "dask[distributed]"
2024-11-12 22:53:04,312 - distributed.worker - INFO -       Start worker at:    tcp://10.142.0.63:34821
2024-11-12 22:53:04,312 - distributed.worker - INFO -          Listening to:    tcp://10.142.0.63:34821
2024-11-12 22:53:04,312 - distributed.worker - INFO -          dashboard at:          10.142.0.63:41265
2024-11-12 22:53:04,312 - distributed.work

KeyboardInterrupt: 